In [1]:
!pip install requests
!pip install tabulate>=0.9 networkx>=3.0
!pip install tsplib95 --no-deps
!pip install deprecated


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


# Importe de librerias

In [2]:
from typing import Union, Tuple, List, Literal, Dict, Any
import random
import gzip
import shutil
import os
import urllib.request
from copy import deepcopy

import tsplib95

## Métodos Auxiliares y algunas definiciones necesarias


In [3]:
NumericType = Union[int, float]
NodeType = int
EdgeType = Tuple[NodeType, NodeType]
SolutionType = List[NodeType]
TSProblemType = tsplib95.models.Problem

def download_tsp_file(url: str, filename: str) -> None:
    # Descarga como archivo temporal
    local_path, headers = urllib.request.urlretrieve(url, filename + ".temp")
    mime_type = headers.get_content_type()
    # Descomprimir si es formato gzip
    if "gzip" in mime_type or local_path.endswith(".gz"):
        with gzip.open(local_path, 'rb') as f_in:
            with open(filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        os.remove(local_path)
    # Caso contrario renombra el archivo temporal
    else:
        os.replace(local_path, filename)

In [4]:
#DATOS DEL PROBLEMA
# Matriz Adyacencia swiss42 problem (Staedte Schweiz/Fricker)
file = "swiss42.tsp"
download_tsp_file("https://raw.githubusercontent.com/mastqe/tsplib/refs/heads/master/swiss42.tsp", file)
tsp_problem: tsplib95.models.Problem = tsplib95.load(file)

#Nodos
nodes = list(tsp_problem.get_nodes())

# Funcionas básicas

In [5]:
def distancia(a: int, b: int, problem: TSProblemType) -> NumericType:
    """
    Devuelve la distancia entre dos nodos
    :param a: Nodo a
    :param b: Nodo b
    :param problem: Instancia de tsplib95.models.Problem
    :return:
    """
    return problem.get_weight(a, b)

def distancia_total(solucion: List[int], problem: TSProblemType) -> NumericType:
    """
    Devuelve la distancia total de una trayectoria/solución
    :param solucion: Lista de Nodos
    :param problem: Instancia de tsplib95.models.Problem
    :return:
    """
    distancia_total = 0
    n = len(solucion)
    for i in range(len(solucion)):
        distancia_total += distancia(solucion[i], solucion[(i + 1)%n], problem)
    return distancia_total

## Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser más eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$$
p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in
J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}, \text{ si }j \in J^k_i
$$

$$p^k_{ij}(t) = 0, \text{ si }j \notin J^k_i$$

In [6]:
def add_node(
        problem: TSProblemType,
        H: SolutionType,
        T: List[List[NumericType]]
) -> NumericType:
    """
    Mejora: Establecer una función de probabilidad para añadir un nuevo nodo
    dependiendo de los nodos más cercanos y de las feromonas depositadas
    :param problem:
    :param H:
    :param T:
    :return:
    """
    nodos = list(problem.get_nodes())
    return random.choice(list(set(range(1, len(nodos))) - set(H)))

def increase_pheromone(
        problem: TSProblemType,
        T: List[List[NumericType]],
        H: SolutionType,
        q_factor: float = 1000.0
) -> List[List[NumericType]]:
    """
    Incrementa según la calidad de la solución. Añadir una cantidad
    inversamente proporcional a la distancia total
    :param problem:
    :param T:
    :param H:
    :param q_factor:
    :return:
    """
    n = len(H)
    distancia_recorrida = distancia_total(H, problem)
    delta = q_factor / distancia_recorrida
    for i in range(n):
        T[H[i]][H[(i + 1)%n]] += delta
    return T


def evaporate_pheromones(T: List[List[NodeType]]) -> List[List[NumericType]]:
    """
    Evapora 0.3 el valor de la feromona, sin que baje de 1\n
    Mejora: Podemos elegir diferentes funciones de evaporación dependiendo
    de la cantidad actual y de la suma total de feromonas depositadas,...
    :param T:
    :return:
    """
    T = [
        [max(T[i][j] - 0.3, 1) for i in range(len(nodes))]
        for j in range(len(nodes))
    ]
    return T

def init_pheromones(problem: TSProblemType) -> List[List[NumericType]]:
    """
    Inicializa las aristas con una cantidad inicial de feromonas:1\n
    Mejora: inicializar con valores diferentes dependiendo diferentes criterios
    :param n:
    :return:
    """
    nodes = list(problem.get_nodes())
    m = len(nodes)
    return [[1 for _ in range(m)] for _ in range(m)]

## Mejoras propuestas

In [7]:
def add_node_uniform(
        problem: TSProblemType,
        H: SolutionType,
        T: List[List[NumericType]],
        alpha: float = 1.0,
        beta: float = 2.0,
        q0: float = 0.0
) -> NumericType:
    current_node = H[-1]
    n_nodes = len(list(problem.get_nodes()))
    unvisited = list(set(range(n_nodes)) - set(H))
    if not unvisited:
        return None
    weights = []
    for candidate in unvisited:
        dist = problem.get_weight(current_node, candidate)
        eta = 1.0/(dist + 1e-6)
        tau = T[current_node][candidate]
        w = (tau ** alpha) * (eta ** beta)
        weights.append(w)
    if random.random() < q0:
        return unvisited[weights.index(max(weights))]
    else:
        return random.choices(unvisited, weights=weights, k=1)[0]

def evaporate_pheromones_exponential_decay(
        T: List[List[NumericType]],
        rho: float = 0.1,
        tau_min: float = 0.01
) -> List[List[NumericType]]:
    decay_factor = 1.0 - rho
    n = len(T)
    for i in range(n):
        for j in range(n):
            T[i][j] = max(T[i][j]*decay_factor, tau_min)
    return T

def evaporate_pheromones_adaptative(
    T: List[List[float]],
    tau_min: float = 0.01,
    tau_max: float = 100.0
) -> List[List[NumericType]]:
    n = len(T)
    m = len(T[0])
    total_pheromones = sum(sum(row) for row in T)
    avg_pheromone = total_pheromones / (n * m)
    decay = 1.0 - min(0.5, max(0.5, avg_pheromone / tau_max))
    for i in range(n):
        for j in range(n):
            new_val = T[i][j] * decay
            if new_val < tau_min:
                T[i][j] = tau_min
            elif new_val > tau_max:
                T[i][j] = tau_max
            else:
                T[i][j] = new_val
    return T

def init_pheromones_adaptative(problem: TSProblemType) -> List[List[NumericType]]:
    nodes = list(problem.get_nodes())
    m = len(nodes)

    current_node = nodes[0]
    unvisited = set(nodes[1:])
    path_cost = 0.0

    while unvisited:
        next_node = min(unvisited, key=lambda x: problem.get_weight(current_node, x))
        path_cost += problem.get_weight(current_node, next_node)
        unvisited.remove(next_node)
        current_node = next_node
    path_cost += problem.get_weight(current_node, nodes[0])
    tau0 = 1.0 / (m * path_cost)
    return [[tau0 for _ in range(m)] for _ in range(m)]

init_pheromones_strategies = {
    "base": init_pheromones,
    "adaptative": init_pheromones_adaptative,
}

add_node_strategies = {
    "base": add_node,
    "uniform": add_node_uniform
}

evaporate_pheromones_strategies = {
    "base": evaporate_pheromones,
    "exponential": evaporate_pheromones_exponential_decay,
    "adaptative": evaporate_pheromones_adaptative,
}

In [8]:
def hormigas(
        problem: TSProblemType,
        n_hormigas: int,
        add_node_strategy: Literal["base", "uniform"] = "base",
        add_node_kwargs: Dict[str, Any] = {},
        init_pheromones_strategy: Literal["base", "adaptative"] = "base",
        init_pheromones_kwargs: Dict[str, Any] = {},
        evaporate_pheromones_strategy: Literal["base", "exponential", "adaptative"] = "base",
        evaporate_pheromones_kwargs: Dict[str, Any] = {},
) -> Tuple[SolutionType, NumericType]:
    """
    :param problem: datos del problema
    :param n_hormigas: Número de agentes(hormigas)
    :param add_node_strategy:
    :param init_pheromones_strategy:
    :param evaporate_pheromones_strategy:
    :return:
    """

    #Nodos
    nodes = list(problem.get_nodes())
    n_cities = len(nodes)

    init_pheromones_callback = init_pheromones_strategies.get(init_pheromones_strategy, "base")
    add_node_callback = add_node_strategies.get(add_node_strategy, "base")
    evaporate_pheromones_callback = evaporate_pheromones_strategies.get(evaporate_pheromones_strategy, "base")

    #Inicializa las aristas con una cantidad inicial de feromonas
    T = init_pheromones_callback(problem, **init_pheromones_kwargs)

    #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
    ants = [[0] for _ in range(n_hormigas)]

    #Recorre cada agente construyendo la solución
    for h in range(n_hormigas):
        #Para cada agente se construye un camino
        for i in range(n_cities - 1):
            #Elige el siguiente nodo
            new_node = add_node_callback(problem, ants[h], T, **add_node_kwargs)
            ants[h].append(new_node)

        #Incrementa feromonas en esa arista
        T = increase_pheromone(problem, T, ants[h])
        #print("Feromonas(1)", T)

        #Evapora Feromonas
        T = evaporate_pheromones_callback(T, **evaporate_pheromones_kwargs)
        #print("Feromonas(2)", T)

        #Seleccionamos el mejor agente
    mejor_solucion = []
    mejor_distancia = float('inf')
    for h in range(n_hormigas):
        distancia_actual = distancia_total(ants[h], problem)
        if distancia_actual < mejor_distancia:
            mejor_solucion = ants[h]
            mejor_distancia = distancia_actual

    print(mejor_solucion)
    print(mejor_distancia)
    return mejor_solucion, mejor_distancia

solucion_actual, distancia_actual = hormigas(tsp_problem, 1000)

[0, 7, 28, 31, 5, 4, 8, 24, 14, 3, 19, 23, 22, 38, 34, 33, 32, 41, 25, 29, 12, 16, 20, 17, 18, 26, 11, 2, 27, 1, 37, 6, 9, 39, 21, 10, 40, 30, 15, 35, 13, 36]
3909


In [9]:
solucion_actual, distancia_actual = hormigas(
    tsp_problem, 1000,
    add_node_strategy="uniform",
    # add_node_kwargs={"q0": 0.9},
    init_pheromones_strategy="adaptative",
    evaporate_pheromones_strategy="exponential"
)

[0, 1, 6, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 39, 21, 40, 24, 22, 38, 30, 29, 28, 2, 27, 3, 4, 7, 17, 31, 35, 36, 37, 15, 14, 16, 19, 13, 5, 20, 33, 34, 32]
1460


In [10]:
solucion_actual, distancia_actual = hormigas(
    tsp_problem, 1000,
    add_node_strategy="uniform",
    # add_node_kwargs={"q0": 0.9},
    init_pheromones_strategy="adaptative",
    evaporate_pheromones_strategy="adaptative"
)

[0, 1, 6, 26, 5, 19, 7, 14, 16, 15, 37, 17, 31, 35, 36, 4, 2, 27, 3, 28, 30, 29, 32, 20, 33, 34, 38, 24, 22, 39, 21, 40, 9, 23, 41, 8, 10, 25, 11, 12, 18, 13]
1645


In [11]:
def hormigas(
        problem: TSProblemType,
        n_hormigas: int,
        add_node_strategy: Literal["base", "uniform"] = "base",
        add_node_kwargs: Dict[str, Any] = {},
        init_pheromones_strategy: Literal["base", "adaptative"] = "base",
        init_pheromones_kwargs: Dict[str, Any] = {},
        evaporate_pheromones_strategy: Literal["base", "exponential", "adaptative"] = "base",
        evaporate_pheromones_kwargs: Dict[str, Any] = {},
        max_iter: int = 10,
) -> Tuple[SolutionType, NumericType]:
    init_pheromones_callback = init_pheromones_strategies.get(init_pheromones_strategy, "base")
    add_node_callback = add_node_strategies.get(add_node_strategy, "base")
    evaporate_pheromones_callback = evaporate_pheromones_strategies.get(evaporate_pheromones_strategy, "base")
    nodes = list(problem.get_nodes())
    n_cities = len(nodes)
    T = init_pheromones_callback(problem, **init_pheromones_kwargs)
    best_global_solution = []
    best_global_dist = float('inf')
    for iteration in range(max_iter):
        current_gen_ants_paths = []
        for i in range(n_hormigas):
            start_node = random.choice(nodes)
            ant_path = [start_node]
            for _ in range(n_cities - 1):
                new_node = add_node_callback(problem, ant_path, T, **add_node_kwargs)
                ant_path.append(new_node)
            current_gen_ants_paths.append(ant_path)

        T = evaporate_pheromones_callback(T, **evaporate_pheromones_kwargs)
        for path in current_gen_ants_paths:
            distance = distancia_total(path, problem)
            if distance < best_global_dist:
                best_global_dist = distance
                best_global_solution = deepcopy(path)

            T = increase_pheromone(problem, T, best_global_solution)

            # opción elitista
            # if distance < best_global_dist * 1.2:
            #     T = increase_pheromone(problem, T, path)

        # Refuerzo elitista
        # if best_global_solution:
        #     T = increase_pheromone(problem, T, best_global_solution)

    print(best_global_solution)
    print(best_global_dist)
    return best_global_solution, best_global_dist

solucion_actual, distancia_actual = hormigas(
    tsp_problem, 1000,
    add_node_strategy="uniform",
    max_iter=15
)

[21, 39, 22, 38, 34, 33, 20, 32, 30, 29, 28, 2, 27, 3, 4, 6, 1, 0, 7, 17, 31, 35, 36, 37, 15, 14, 16, 19, 13, 26, 5, 18, 12, 11, 25, 10, 8, 41, 23, 9, 40, 24]
1328


In [12]:
solucion_actual, distancia_actual = hormigas(
    tsp_problem, 1000,
    add_node_strategy="uniform",
    # add_node_kwargs={"q0":0.9},
    init_pheromones_strategy="adaptative",
    evaporate_pheromones_strategy="exponential",
    max_iter=30
)

[13, 19, 14, 16, 15, 37, 17, 31, 35, 36, 7, 0, 1, 6, 3, 4, 27, 2, 28, 29, 30, 34, 33, 20, 32, 38, 22, 39, 21, 40, 24, 23, 41, 9, 8, 10, 25, 11, 12, 18, 26, 5]
1390


In [13]:
solucion_actual, distancia_actual = hormigas(
    tsp_problem, 1000,
    add_node_strategy="uniform",
    # add_node_kwargs={"q0":0.9},
    init_pheromones_strategy="adaptative",
    evaporate_pheromones_strategy="adaptative",
    max_iter=30
)

[37, 15, 14, 16, 19, 13, 5, 26, 18, 12, 11, 25, 10, 41, 23, 8, 9, 21, 40, 24, 39, 22, 38, 30, 29, 28, 2, 27, 3, 4, 6, 1, 0, 32, 33, 34, 20, 35, 36, 31, 17, 7]
1311
